In [48]:
import export_objects as eo
import analyze_subroutines as alsub
import fortran_modules as fm
import utilityFunctions as uf
import namelist as nl
import importlib
import os
import csv

import database.connect as cnt
import database.utilities as util

importlib.reload(eo)
importlib.reload(alsub)
importlib.reload(fm)
importlib.reload(uf)
importlib.reload(cnt)
importlib.reload(util)
importlib.reload(nl)

mod_dict, sub_dict, type_dict = {}, {}, {}
mod_dict, sub_dict, type_dict = eo.unpickle_unit_test("-352536b")


In [167]:
from variable_analysis import determine_global_variable_status
sub_name_list = [
        "Allocation1_PlantNPDemand",
        "Allocation2_ResolveNPLimit",
        "Allocation3_PlantCNPAlloc",
    ]

subroutines = {s.lower() : sub_dict[s.lower()] for s in sub_name_list}

glob_vars_test = determine_global_variable_status(mod_dict, subroutines)
print(glob_vars_test)


/mnt/c/users/mungs/Desktop/SPEL_OpenACC/scripts/../../repo/E3SM/components/elm/src/biogeochem/AllocationMod.F90
1:module AllocationMod
integer shr_kind_r8 0


AttributeError: 'str' object has no attribute 'name'

In [143]:
import export_objects as eo
import analyze_subroutines as alsub
import fortran_modules as fm
import utilityFunctions as uf
import namelist as nl
import importlib
import os
import csv

import database.connect as cnt
import database.utilities as util

importlib.reload(eo)
importlib.reload(alsub)
importlib.reload(fm)
importlib.reload(uf)
importlib.reload(cnt)
importlib.reload(util)



from mod_config import ELM_SRC, E3SM_SRCROOT
allocationMod = os.path.join(ELM_SRC, "biogeochem", "AllocationMod.F90")
mod_dict, sub_dict, type_dict = {}, {}, {}
mod_dict, sub_dict, type_dict = eo.unpickle_unit_test("-352536b")


AttributeError: 'DerivedType' object has no attribute 'filepath'

In [64]:
allocationMod

'/mnt/c/users/mungs/Desktop/SPEL_OpenACC/scripts/../../repo/E3SM/components/elm/src/biogeochem/AllocationMod.F90'

In [141]:
importlib.reload(nl)
nl.insert_default(mod_dict)

namelist = nl.find_all_namelist()

all = nl.run(allocationMod)


nl.generate_namelist_dict(mod_dict, namelist, all)
namelist['nu_com'].variable.default_value = "ECA"
print(namelist['nu_com'].variable.default_value)
# nl.set_default(all, namelist)



ECA


In [103]:
print(all[nl.binary_search(all, 1104)].relative_end)
for i in all[nl.binary_search(all, 1104)].elseif: print(i.default, i.condition)


1111
True nu_com .eq. 'ECA' .or. nu_com .eq. 'MIC'


In [140]:
pp = [i for i in all if i.start == 1104]
nl.set_default(pp, namelist)
# pp[0].condition, pp[0].default
x = nl.flatten(pp)
for i in x:
    if i.start == 1113:
        nl.print_if_structure(i)
        print(i.condition, i.default)
        
  

start: 1113, end: 1129, depth: 2, relative: 1119, type: 1
condition: use_fates
default: True
else:
->|start: 1119, end: 1129, depth: 2, relative: 1129, type: 3
->|default: True
->|parent: 1113
parent: 1104
use_fates True


In [110]:
nl.print_if_structure(pp[0])

start: 1104, end: 1131, depth: 1, relative: 1111, type: 1
condition: nu_com .eq. 'RD'
default: False
calls: [(1108, 'calc_nuptake_prof'), (1109, 'calc_puptake_prof')]
else if:
->|start: 1111, end: 1131, depth: 1, relative: 1131, type: 2
->|condition: nu_com .eq. 'ECA' .or. nu_com .eq. 'MIC'
->|default: True
->|parent: 1104
->|children: 
->|->|->|start: 1113, end: 1129, depth: 2, relative: 1119, type: 1
->|->|->|condition: use_fates
->|->|->|default: True
->|->|->|else:
->|->|->|->|start: 1119, end: 1129, depth: 2, relative: 1129, type: 3
->|->|->|->|default: True
->|->|->|->|parent: 1113
->|->|->|parent: 1104
parent: -1


In [101]:
y = nl.elmtypes(sub_dict, pp, "Allocation2_ResolveNPLimit".lower())

1111 1131 1131 False True
removed bounds%begp ReadWrite(status='r', ln=1120)
1111 1131 1131 False True
removed bounds%endp ReadWrite(status='r', ln=1120)
1111 1131 1131 False True
removed veg_nf%smin_nh4_to_plant ReadWrite(status='w', ln=1124)
1111 1131 1131 False True
removed veg_nf%smin_no3_to_plant ReadWrite(status='w', ln=1125)
1111 1131 1131 False True
removed veg_nf%sminn_to_plant ReadWrite(status='w', ln=1126)
1111 1131 1131 False True
removed veg_pf%sminp_to_plant ReadWrite(status='w', ln=1127)


In [61]:
y

{'bounds%begp': [],
 'bounds%endp': [],
 'veg_nf%smin_nh4_to_plant': [ReadWrite(status='rw', ln=1642),
  ReadWrite(status='r', ln=1663)],
 'veg_nf%smin_no3_to_plant': [ReadWrite(status='r', ln=1663),
  ReadWrite(status='rw', ln=1644)],
 'veg_nf%sminn_to_plant': [ReadWrite(status='w', ln=1663)],
 'veg_pf%sminp_to_plant': [ReadWrite(status='rw', ln=1646)],
 'col_pp%landunit': [ReadWrite(status='r', ln=1138)],
 'bounds%clump_index': [ReadWrite(status='r', ln=1149)],
 'col_nf%plant_ndemand': [ReadWrite(status='rw', ln=1611),
  ReadWrite(status='w', ln=1160),
  ReadWrite(status='r', ln=1696),
  ReadWrite(status='r', ln=1695),
  ReadWrite(status='r', ln=1222),
  ReadWrite(status='w', ln=1608),
  ReadWrite(status='r', ln=1761),
  ReadWrite(status='rw', ln=1184)],
 'col_pf%plant_pdemand': [ReadWrite(status='r', ln=1790),
  ReadWrite(status='r', ln=1223),
  ReadWrite(status='w', ln=1609),
  ReadWrite(status='r', ln=1714),
  ReadWrite(status='rw', ln=1185),
  ReadWrite(status='w', ln=1161),
  Re

In [114]:
sub_dict["Allocation2_ResolveNPLimit".lower()].elmtype_access_by_ln['veg_pp%itype']

[ReadWrite(status='r', ln=1234),
 ReadWrite(status='r', ln=1238),
 ReadWrite(status='r', ln=1262),
 ReadWrite(status='r', ln=1262),
 ReadWrite(status='r', ln=1262),
 ReadWrite(status='r', ln=1283),
 ReadWrite(status='r', ln=1283),
 ReadWrite(status='r', ln=1283)]

In [2]:
for i in sub_dict["allocation2_resolvenplimit"].elmtype_access_by_ln.keys():
    if i=="veg_nf%smin_no3_to_plant": 
        print(sub_dict["allocation2_resolvenplimit"].elmtype_access_by_ln[i])

[ReadWrite(status='w', ln=1125), ReadWrite(status='rw', ln=1644), ReadWrite(status='rw', ln=1644), ReadWrite(status='r', ln=1663)]


In [ ]:
for i in namelist:
    if namelist[i].if_blocks:
        
        for j in (namelist[i].if_blocks):
            print(j.default)

BREAKKK


In [ ]:
# connect to database
cur, conn = cnt.connect()

# use file to create csv for module table
csv_folder = "CSV"
info_files = "info_files"
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(info_files, exist_ok=True)

In [ ]:
# write modules into to file
def write_modules_info_to_text(filename):
    with open(filename, "w") as file:
        for keys in mod_dict.keys():
            mod_dict[keys].export_module_info(ofile=file)
            
# write subroutines into to file
def write_subroutines_info_to_text(filename):
    with open(filename, "w") as file:
        for i in sub_dict.keys():
            sub_dict[i].export_subroutine_info(ofile=file)
            
# write types into to file
def write_types_info_to_text(filename):
    with open(filename, "w") as file:
        for i in type_dict.keys():
            type_dict[i].export_derived_type(ofile=file)

In [ ]:
write_modules_info_to_text(f"{info_files}/modules.txt")
write_subroutines_info_to_text(f"{info_files}/subroutine.txt")
write_types_info_to_text(f"{info_files}/types_info.txt")

In [ ]:


def TABLE_modules(filename, csv_file="TABLE_module"):
    
    """
    TABLE SETUP:
    +-------------+--------------+------+-----+---------+----------------+
    | Field       | Type         | Null | Key | Default | Extra          |
    +-------------+--------------+------+-----+---------+----------------+
    | module_id   | int          | NO   | PRI | NULL    | auto_increment |
    | module_name | varchar(100) | NO   | UNI | 0       |                |
    +-------------+--------------+------+-----+---------+----------------+
    
    INITIALIZE NULL WITH COMMAND
    
    INSERT INTO modules (module_name) VALUES ('NULL');
    UPDATE modules SET module_id=0 where module_id=1;
    
    """

    cols = ["module_name"]
    
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as reader:
            for line in reader:
                if "Module:" in line:
                    # Module: <module>
                    module = line.split(" ")[-1].strip()
                    
                    writer.writerow([module])

def TABLE_module_dependency(filename, csv_file="TABLE_module_dependency"):
    
    """
    TABLE SETUP
    +---------------+--------------+------+-----+---------+----------------+
    | Field         | Type         | Null | Key | Default | Extra          |
    +---------------+--------------+------+-----+---------+----------------+
    | dependency_id | int          | NO   | PRI | NULL    | auto_increment |
    | module_id     | int          | YES  | MUL | NULL    |                |
    | dep_module_id | int          | YES  | MUL | NULL    |                |
    | object_used   | varchar(100) | YES  |     | NULL    |                |
    +---------------+--------------+------+-----+---------+----------------+
    """
    
    cols = ["module_id", "dep_module_id","object_used"]

    with open(f"{csv_folder}/{csv_file}.csv","w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, "r") as reader:
            cur_module = ""
            cur_dep = ""
            for line in reader:
                if "Module:" in line:
                    # Module: <module>
                    cur_module = line.split(" ")[-1].strip()
                elif "->" in line:
                    if "None" in line:
                        writer.writerow([cur_module, "NULL", "NULL"])
                        continue
                    cur_dep = line.split(" ")[-1].strip()
                elif line[:4] == "   -":
                    line = line.strip().split()[1:]
                    if "=>" in line:
                        line = line[2:]
                    if len(line) == 1:
                        writer.writerow([cur_module, cur_dep, line[0].strip()])

                    if len(line) == 3:
                        writer.writerow([cur_module, cur_dep, line[1].strip()])
                    

In [ ]:
filename = f"{info_files}/modules.txt"

TABLE_modules(filename)
TABLE_module_dependency(filename)

util.insert(cur,
            conn,
            "modules",
            "CSV/TABLE_module.csv")

util.insert(cur, 
            conn, 
            "module_dependency", 
            "CSV/TABLE_module_dependency.csv", 
            join=True, 
            ref_tables=["modules", "modules", 2],
            selections=["module_id","module_id",""],
            conditions=["module_name","module_name"],
            dependency=[0,1,None])

In [ ]:
def TABLE_user_types(filename, csv_file='TABLE_user_types'):
    """
    TABLE SETUP
    +----------------+--------------+------+-----+---------+----------------+
    | Field          | Type         | Null | Key | Default | Extra          |
    +----------------+--------------+------+-----+---------+----------------+
    | user_type_id   | int          | NO   | PRI | NULL    | auto_increment |
    | module_id      | int          | NO   | MUL | 0       |                |
    | user_type_name | varchar(100) | NO   | UNI | 0       |                |
    +----------------+--------------+------+-----+---------+----------------+
    """
    cols = ["module_id", "user_type_name"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, 'r') as f:
            type = ""
            mod = ""
            for line in f:
                if "Type:" in line:
                    type = line.split()[-1].strip()
                elif "Mod:" in line:
                    mod = line.split()[-1].strip()
                    writer.writerow([mod, type])

def TABLE_user_type_instances(filename, csv_file='TABLE_user_type_instances'):
    
    """
    TABLE SETUP
    +------------------+--------------+------+-----+---------+----------------+
    | Field            | Type         | Null | Key | Default | Extra          |
    +------------------+--------------+------+-----+---------+----------------+
    | instance_id      | int          | NO   | PRI | NULL    | auto_increment |
    | instance_type_id | int          | NO   | MUL | 0       |                |
    | instance_name    | varchar(100) | NO   |     |         |                |
    +------------------+--------------+------+-----+---------+----------------+
    """
    
    cols = ["instance_type_id", "instance_name"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, 'r') as f:
            type = ""
            
            for line in f:
                if "Type:" in line:
                    type = line.split()[-1].strip()
                elif "i:" in line:
                    instance = line.split()[2].strip()
                    writer.writerow([type, instance])

In [ ]:
filename = f"{info_files}/types_info.txt"

TABLE_user_types(filename)
TABLE_user_type_instances(filename)

util.insert(cur, 
            conn, 
            "user_types", 
            "CSV/TABLE_user_types.csv", 
            join=True, 
            ref_tables=["modules", 1],
            selections=["module_id", ""],
            conditions=["module_name", ""],
            dependency=[0,None])

util.insert(cur,
            conn,
            "user_type_instances",
            "CSV/TABLE_user_type_instances.csv",
            join=True,
            ref_tables=["user_types", 1],
            selections=["user_type_id", ""],
            conditions=['user_type_name', ""],
            dependency=[0,None])

In [ ]:
def TABLE_type_defines(filename, csv_file="TABLE_type_defines"):
    with open(csv_file, 'w') as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(['subroutine_parent', "module_parent", "subroutine_child", "module_child", "arg"])

In [ ]:
def TABLE_subroutines(filename, csv_file="TABLE_subroutines"):
    """
    TABLE SETUP
    +-----------------+--------------+------+-----+---------+----------------+
    | Field           | Type         | Null | Key | Default | Extra          |
    +-----------------+--------------+------+-----+---------+----------------+
    | subroutine_id   | int          | NO   | PRI | NULL    | auto_increment |
    | subroutine_name | varchar(100) | NO   | UNI | 0       |                |
    | module_id       | int          | NO   | MUL | 0       |                |
    +-----------------+--------------+------+-----+---------+----------------+
    """
    cols = ["subroutine_name", "module_id"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as r:
            sub = ""
            mod = ""
            for line in r:
                if "Sub:" in line:
                   sub = line.split()[-1].strip() 
                elif "Mod:" in line:
                    mod = line.split()[-1].strip()
                    writer.writerow([sub, mod])

def TABLE_subroutines_args(filename, csv_file="TABLE_subroutine_args"):
    """
    TABLE SETUP
    +---------------+--------------+------+-----+---------+----------------+
    | Field         | Type         | Null | Key | Default | Extra          |
    +---------------+--------------+------+-----+---------+----------------+
    | arg_id        | int          | NO   | PRI | NULL    | auto_increment |
    | subroutine_id | int          | NO   | MUL | 0       |                |
    | arg_type      | varchar(100) | NO   |     | 0       |                |
    | arg_name      | varchar(100) | NO   |     | 0       |                |
    | dim           | int          | NO   |     | 0       |                |
    +---------------+--------------+------+-----+---------+----------------+
    """
    cols = ["subroutine_id", "arg_type", "arg_name", "dim"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as r:
            sub = ""
            for line in r:
                if "Sub:" in line:
                   sub = line.split()[-1].strip() 
                elif "-" in line:
                    if "None" in line:
                        writer.writerow([sub, "Null", "Null", "-1"])
                        continue
                    line = line.split()
                    line = [l.strip() for l in line]
                 
                    writer.writerow([sub, line[1], line[-2].strip("'"), line[-1]])

def TABLE_subroutines_calltree(filename, csv_file="TABLE_subroutine_calltree"):
    """
    TABLE SETUP
    +-----------------------+------+------+-----+---------+----------------+
    | Field                 | Type | Null | Key | Default | Extra          |
    +-----------------------+------+------+-----+---------+----------------+
    | parent_id             | int  | NO   | PRI | NULL    | auto_increment |
    | parent_subroutine_id  | int  | NO   | UNI | 0       |                |
    | child_subroutine_id   | int  | NO   | MUL | 0       |                |
    +-----------------------+------+------+-----+---------+----------------+
    """
    cols = ["parent_subroutine_id", "child_subroutine_id"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as r:
            sub = ""
            for line in r:
                if "ParentSub:" in line:
                   sub = line.split()[-1].strip() 
                elif "c:" in line:
                    if "None" in line:
                        writer.writerow([sub, "Null"])
                        continue
                    else:
                        line = line.split()[-1].strip()
                    
                    
                        writer.writerow([sub, line])



def TABLE_subroutine_active_global_vars(filename= "CSV/_TABLE_subroutine_active_global_vars.csv",
                                        csv_file="TABLE_subroutine_active_global_vars"):
    """
    TABLE SETUP
    +---------------+---------+------+-----+---------+----------------+
    | Field         | Type    | Null | Key | Default | Extra          |
    +---------------+---------+------+-----+---------+----------------+
    | variable_id   | int     | NO   | PRI | NULL    | auto_increment |
    | subroutine_id | int     | NO   | MUL | 0       |                |
    | instance_id   | int     | YES  | MUL | NULL    |                |
    | member_id     | int     | NO   | MUL | 0       |                |
    | status        | char(2) | NO   |     | 0       |                |
    +---------------+---------+------+-----+---------+----------------+
    """
    eo.create_dataframe(sub_dict, "CSV/_TABLE_subroutine_active_global_vars")
    
    cols = ["subroutine_id", "instance_id", "member_id", "status"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, "r") as r:
            header = True
            reader = csv.reader(r)
            for line in reader:
                if header:
                    header = False
                    continue
                # print(line)
                line = [l.strip() for l in line]
                var = line[2].split('%')
                instance = var[0]
                member = var[1]
                writer.writerow([line[1], instance, member, line[3]])

def TABLE_subroutines_local_array(filename, csv_file="TABLES_subroutine_local_array"):
    """
    TABLE SETUP
    +---------------+--------------+------+-----+---------+----------------+
    | Field         | Type         | Null | Key | Default | Extra          |
    +---------------+--------------+------+-----+---------+----------------+
    | local_arry_id | int          | NO   | PRI | NULL    | auto_increment |
    | subroutine_id | int          | NO   | MUL | 0       |                |
    | array_name    | varchar(100) | NO   |     | 0       |                |
    | dim           | int          | NO   |     | 0       |                |
    +---------------+--------------+------+-----+---------+----------------+
    """
    cols = ["subroutine_id", "array_name", "dim"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as r:
            sub = ""
            for line in r:
                if "ParentSub:" in line:
                   sub = line.split()[-1].strip() 
                elif "l:" in line:
                    if "None" in line:
                        writer.writerow([sub, "Null", "-1"])
                        continue
                    else:
                        line = line.split()
                        line = [l.strip() for l in line]
                    
                    
                        writer.writerow([sub, line[2], line[-1]])

In [ ]:
TABLE_subroutines_calltree()
util.insert(cur,
            conn,
            "subroutine_calltree",
            "CSV/TABLE_subroutine_calltree.csv",
            join=True,
            ref_tables=['subroutines', 'subroutines'],
            selections=['subroutine_id', 'subroutine_id'],
            conditions=['subroutine_name', 'subroutine_name'],
            dependency=[0, 1])


In [ ]:
import export_objects as eo
import analyze_subroutines as alsub
import fortran_modules as fm
import utilityFunctions as uf
import importlib
import os
import csv

import database.connect as cnt
import database.utilities as util
# 
importlib.reload(eo)
importlib.reload(alsub)
importlib.reload(fm)
importlib.reload(uf)
importlib.reload(cnt)
importlib.reload(util)
#
mod_dict, sub_dict, type_dict = {}, {}, {}
mod_dict, sub_dict, type_dict = eo.unpickle_unit_test(mod_dict, sub_dict, type_dict)
cur, conn = cnt.connect()


In [ ]:


# TABLE_subroutine_active_global_vars()
util.insert(cur,
            conn,
            "subroutine_active_global_vars",
            "CSV/TABLE_subroutine_active_global_vars.csv",
            join=True,
                # cols = ["subroutine_id", "instance_id", "member_id", "status"]
            ref_tables=['subroutines', "user_type_instances", 'type_definitions', 3],
            selections=['subroutine_id', 'instance_id', "define_id", ""],
            conditions=['subroutine_name', 'instance_name', 'member_name'],
            dependency=[0, 1, 2, None],
            extra=["m1.instance_type_id=m2.user_type_id"])

In [ ]:
filename = f"{info_files}/subroutine.txt"

TABLE_subroutines(filename)
TABLE_subroutines_args(filename)
TABLE_subroutines_calltree(filename)
TABLE_subroutine_active_global_vars()
TABLE_subroutines_local_array(filename)

util.insert(cur,
            conn,
            "subroutines",
            "CSV/TABLE_subroutines.csv",
            join=True,
            ref_tables=[0, "modules"],
            selections=["","module_id"],
            conditions=["","module_name"],
            dep_cols=[None,1])

util.insert(cur,
            conn,
            "subroutine_args",
            "CSV/TABLE_subroutine_args.csv",
            join=True,
            ref_table=["subroutines", 1, 2, 3],
            selections=["subroutine_id", "", "", ""],
            conditions=["subroutine_name", "", "", ""],
            dependency=[0, None, None, None])

util.insert(cur,
            conn,
            "subroutine_calltree",
            "CSV/TABLE_subroutine_calltree.csv",
            join=True,
            ref_tables=['subroutines', 'subroutines'],
            selections=['subroutine_id', 'subroutine_id'],
            conditions=['subroutine_name', 'subroutine_name'],
            dependency=[0, 1])

util.insert(cur,
            conn,
            "subroutine_active_global_vars",
            "CSV/TABLE_subroutine_active_global_vars.csv",
            join=True,
            ref_tables=['subroutines', "user_type_instances", 2, 3],
            selections=['subroutine_id', 'instance_type_id', "", ""],
            conditions=['subroutine_name', 'instance_name'],
            dependency=[0, 1, None, None])

util.insert(cur,
            conn,
            "subroutine_local_arrays",
            "CSV/TABLEs_subroutine_local_array.csv",
            join=True,
            ref_tables=['subroutines', 1,2 ],
            selections=['subroutine_id', '', ""],
            conditions=['subroutine_name', '', ""],
            dependency=[0, None, None])

In [ ]:
import re
def TABLE_type_definitions(filename, csv_file='TABLE_type_definitions'):    
    """
    TABLE SETUP
    +--------------+--------------+------+-----+---------+----------------+
    | Field        | Type         | Null | Key | Default | Extra          |
    +--------------+--------------+------+-----+---------+----------------+
    | define_id    | int          | NO   | PRI | NULL    | auto_increment |
    | module_id    | int          | NO   | MUL | 0       |                |
    | user_type_id | int          | NO   | MUL | NULL    |                |
    | member_type  | varchar(100) | NO   |     |         |                |
    | member_name  | varchar(100) | NO   |     | 0       |                |
    | dim          | int          | NO   |     | 0       |                |
    | bounds       | varchar(100) | NO   |     |         |                |
    | active       | varchar(100) | YES  |     | NULL    |                |
    +--------------+--------------+------+-----+---------+----------------+
    """
    cols = ["module_id", "user_type_id", "member_type", "member_name", "dim", "bounds", "active"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, 'r') as f:
            type = ""
            mod = ""
            for line in f:
                if "Type:" in line:
                    type = line.split()[-1].strip()
                elif "Mod:" in line:
                    mod = line.split()[-1].strip()
                elif "-" in line:
 
                    x = re.findall(r'[^()\s]+|\(.*?\)', line)
                    if len(x) == 2:
                        writer.writerow([mod, type, "Null", "Null", "-1", "Null", "Null"])
                    else:
                        writer.writerow([mod, type, x[1], x[2], x[4], x[3], x[5]])


In [ ]:
TABLE_type_definitions("info_files/types_info.txt")


In [ ]:
filename = f"{info_files}/types_info.txt"

TABLE_type_definitions("info_files/types_info.txt")

util.insert(cur,
            conn,
            "type_definitions",
            "CSV/TABLE_type_definitions.csv",
            join=True,
            ref_tables=['modules', 'user_types',2, 3, 4, 5, 6 ],
            selections=['module_id', 'user_type_id', "", "", "", "", ""],
            conditions=['module_name', 'user_type_name', "", "", "", "", ""],
            dependency=[0, 1, None, None, None, None, None]
            )